<a href="https://colab.research.google.com/github/dchow2201/Dev-Data-Prod-Slidify/blob/master/Copy_of_Nougat_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq nougat-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 kB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
!pip -q install langchain openai tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


## Read the paper using Nougat model from Meta

In [ ]:
from nougat.utils.checkpoint import get_checkpoint
CHECKPOINT = get_checkpoint('nougat')

downloading nougat checkpoint version 0.1.0-small to path nougat


config.json: 100%|██████████| 557/557 [00:00<00:00, 459kb/s]
pytorch_model.bin: 100%|██████████| 956M/956M [00:02<00:00, 374Mb/s]
special_tokens_map.json: 100%|██████████| 96.0/96.0 [00:00<00:00, 359kb/s]
tokenizer.json: 100%|██████████| 2.04M/2.04M [00:00<00:00, 27.5Mb/s]
tokenizer_config.json: 100%|██████████| 106/106 [00:00<00:00, 131kb/s]


In [ ]:
import subprocess
import uuid
import requests
import re

# Download pdf from a given link
def get_pdf(pdf_link):
  # Generate a unique filename
  unique_filename = f"/content/input/downloaded_paper_{uuid.uuid4().hex}.pdf"

  # Send a GET request to the PDF link
  response = requests.get(pdf_link)

  if response.status_code == 200:
      # Save the PDF content to a local file
      with open(unique_filename, 'wb') as pdf_file:
          pdf_file.write(response.content)
      print("PDF downloaded successfully.")
  else:
      print("Failed to download the PDF.")
  return unique_filename


# Run nougat on the pdf file
def nougat_ocr(file_name):

  # Command to run
  cli_command = [
      'nougat',
      '--out', 'output',
      'pdf', file_name,
      '--checkpoint', CHECKPOINT,
      '--markdown'
  ]

  # Run the command
  subprocess.run(cli_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

  return


# predict function / driver function
def paper_read(pdf_file=None, pdf_link=None):
  if pdf_file is None:
    if pdf_link == '':
      print("No file is uploaded and No link is provided")
      return "No data provided. Upload a pdf file or provide a pdf link and try again!"
    else:
      file_name = get_pdf(pdf_link)
  else:
    file_name = pdf_file

  nougat_ocr(file_name)

  # Open the file for reading
  file_name = file_name.split('/')[-1][:-4]
  with open(f'/content/output/{file_name}.mmd', 'r') as file:
      content = file.read()

  return content


In [ ]:
import os
os.makedirs('input', exist_ok=True)
os.makedirs('output', exist_ok=True)

In [ ]:
%%time
content = paper_read("/content/input/2205.14135.pdf")
print(content)

# FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness

Tri Dao

Department of Computer Science, Stanford University

Daniel Y. Fu

Department of Computer Science and Engineering, University at Buffalo, SUNY

Stefano Ermon

Department of Computer Science, Stanford University

Atri Rudra

Department of Computer Science and Engineering, University at Buffalo, SUNY

Christopher Re

Department of Computer Science and Engineering, University at Buffalo, SUNY

###### Abstract

Transformers are slow and memory-hungry on long sequences, since the time and memory complexity of self-attention are quadratic in sequence length. Approximate attention methods have attempted to address this problem by trading off model quality to reduce the compute complexity, but often do not achieve wall-clock speedup. We argue that a missing principle is making attention algorithms _IO-aware_--accounting for reads and writes between levels of GPU memory. We propose FlashAttention, an IO-awar

In [ ]:
# Specify the filename
filename = "/content/flash_attention.txt"

# Open the file in write mode ('w') and write the string to it
with open(filename, 'w') as file:
    file.write(content)

In [ ]:
%%time
content = paper_read(None, 'https://arxiv.org/pdf/2309.03883v1.pdf')
print(content)

PDF downloaded successfully.
# DoLa: Decoding by Contrasting Layers Improves Factuality in Large Language Models

Yung-Sung Chuang\({}^{\dagger}\)\({}^{\star}\), Yujia Xie\({}^{\ddagger}\), Hongyin Luo\({}^{\dagger}\), Yoon Kim\({}^{\dagger}\), James Glass\({}^{\dagger}\), Pengcheng He\({}^{\ddagger}\)

\({}^{\dagger}\)Massachusetts Institute of Technology, \({}^{\ddagger}\)Microsoft

yungsung@mit.edu, yujiaxie@microsoft.com

{hyluo,yoonkim,glass}@mit.edu

herbert.he@gmail.com

Work done during an internship at Microsoft.

###### Abstract

Despite their impressive capabilities, large language models (LLMs) are prone to hallucinations, i.e., generating content that deviates from facts seen during pretraining. We propose a simple decoding strategy for reducing hallucinations with pretrained LLMs that does not require conditioning on retrieved external knowledge nor additional fine-tuning. Our approach obtains the next-token distribution by contrasting the differences in logits obtained f

In [ ]:
## Remove references
import re
refs_re = re.compile(r'(References|REFERENCES)')
noref_content = refs_re.split(content)[0]
print(len(noref_content), len(content))

38083 64593


In [ ]:
noref_content

'# DoLa: Decoding by Contrasting Layers Improves Factuality in Large Language Models\n\nYung-Sung Chuang\\({}^{\\dagger}\\)\\({}^{\\star}\\), Yujia Xie\\({}^{\\ddagger}\\), Hongyin Luo\\({}^{\\dagger}\\), Yoon Kim\\({}^{\\dagger}\\), James Glass\\({}^{\\dagger}\\), Pengcheng He\\({}^{\\ddagger}\\)\n\n\\({}^{\\dagger}\\)Massachusetts Institute of Technology, \\({}^{\\ddagger}\\)Microsoft\n\nyungsung@mit.edu, yujiaxie@microsoft.com\n\n{hyluo,yoonkim,glass}@mit.edu\n\nherbert.he@gmail.com\n\nWork done during an internship at Microsoft.\n\n###### Abstract\n\nDespite their impressive capabilities, large language models (LLMs) are prone to hallucinations, i.e., generating content that deviates from facts seen during pretraining. We propose a simple decoding strategy for reducing hallucinations with pretrained LLMs that does not require conditioning on retrieved external knowledge nor additional fine-tuning. Our approach obtains the next-token distribution by contrasting the differences in lo

As shown above the model has converted the paper to Latex. Typical PDF readers convert the PDF to text where all information about the tables and mathematical equations is lost

## Summarize all the tables in the Parsed Content

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"



In [ ]:
import tiktoken
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    # encoding = tiktoken.get_encoding(encoding_name)
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(noref_content, "gpt-3.5-turbo")

10150

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

context_template="You are a helpful AI Researcher that specializes in analysing research paper outputs presented to you in Latex"

system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)


## Example 1

In [ ]:
human_template= """
Please extract all tables referenced in this paper. The tables are in Latex format. Summarize the tables one by one.
Each summary should be 5-7 sentences long. Include numbers in summary where you can. Make a dictionary with table number, table name and summary of that table.

 PAPER: {paper_content}

 """

In [ ]:
human_message_prompt = HumanMessagePromptTemplate(
            prompt=PromptTemplate(
            template=human_template,
            input_variables=["paper_content"],))

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k",
                  temperature=0.2)

In [ ]:
summary_chain = LLMChain(llm=chat, prompt=chat_prompt_template)

In [ ]:
%%time
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = summary_chain.run(noref_content)

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(f"Calc Total Cost (USD): ${(cb.prompt_tokens/1000)*0.003 + (cb.completion_tokens/1000)*0.004}")

Total Tokens: 10808
Prompt Tokens: 10249
Completion Tokens: 559
Total Cost (USD): $0.032983000000000005
Calc Total Cost (USD): $0.032983000000000005
CPU times: user 118 ms, sys: 13 ms, total: 131 ms
Wall time: 9.31 s


In [ ]:
import pprint
pprint.pprint(output)


('Dictionary of Tables:\n'
 '\n'
 'Table 1:\n'
 '- Table Number: 1\n'
 '- Table Name: Multiple choices results on the TruthfulQA and FACTOR '
 'datasets\n'
 '- Summary: This table presents the results of multiple choice tasks on the '
 'TruthfulQA and FACTOR datasets. It compares the performance of different '
 'models, including the baseline, Inference Time Intervention (ITI), and DoLa. '
 'The table shows that DoLa consistently outperforms the other methods, '
 'improving the truthfulness and informativeness scores.\n'
 '\n'
 'Table 2:\n'
 '- Table Number: 2\n'
 '- Table Name: Open-ended generation results on TruthfulQA, StrategyQA, and '
 'GSM8K\n'
 '- Summary: This table summarizes the results of open-ended generation tasks '
 'on the TruthfulQA, StrategyQA, and GSM8K datasets. It compares the '
 'performance of different models, including the baseline, Contrastive '
 'Decoding (CD), and DoLa. The table shows that DoLa consistently enhances the '
 'truthfulness and informativeness 

## Example 2

In [ ]:
human_template= """
Please extract all math equations referenced in this paper. The math equation will be in latex. Understand the equation, analyze it and explain it.
Each explanation should be 5-7 sentences long. Make a dictionary with Equation number, Equation in latex and its explanation.

 PAPER: {paper_content}

 """

In [ ]:
human_message_prompt = HumanMessagePromptTemplate(
            prompt=PromptTemplate(
            template=human_template,
            input_variables=["paper_content"],))

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k",
                  temperature=0.2)

summary_chain = LLMChain(llm=chat, prompt=chat_prompt_template)

output_math = summary_chain.run(noref_content)

pprint.pprint(output_math)


('Equation 1: \\(\\hat{p}(x_{t}\\mid x_{<t}) '
 '=\\mathrm{softmax}\\big{(}\\mathcal{F}\\big{(}q_{N}(x_{t}),q_{M}(x_{t '
 '})\\big{)}\\big{)}_{x_{t}}\\)\n'
 '\n'
 'Explanation: This equation represents the probability distribution of the '
 'next token \\(x_{t}\\) given the previous tokens \\(x_{<t}\\). The '
 'probability distribution is obtained by applying the softmax function to the '
 'contrastive difference \\(\\mathcal{F}\\) between the output distributions '
 'of the mature layer \\(q_{N}(x_{t})\\) and the premature layer '
 '\\(q_{M}(x_{t})\\). The contrastive difference is computed in the log '
 'domain, and the softmax function is applied to obtain a normalized '
 'probability distribution over the vocabulary set.\n'
 '\n'
 'Equation 2: '
 '\\(M=\\arg\\max_{j\\in\\mathcal{J}}\\text{JSD}\\big{(}q_{N}(\\cdot\\,|\\,x_{<t})||q_{j}(\\cdot\\,|\\,x_{<t})\\big{)}\\)\n'
 '\n'
 'Explanation: This equation represents the dynamic selection of the premature '
 'layer \\(M\\) based on the